### Import some useful libraries

In [1]:
import pandas as pd
import numpy as np
import datetime

from sklearn import ensemble,model_selection,preprocessing,feature_selection,metrics
import xgboost as xgb
import lightgbm as lgb
import scipy.stats as st

C:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Import the files as dataframes

In [2]:
df_air_reserve = pd.read_csv('air_reserve.csv.zip')
df_air_store = pd.read_csv('air_store_info.csv.zip')
df_air_visit = pd.read_csv('air_visit_data.csv.zip')
df_hpg_reserve = pd.read_csv('hpg_reserve.csv.zip')
df_hpg_store = pd.read_csv('hpg_store_info.csv.zip')
df_date_info = pd.read_csv('date_info.csv.zip')
df_store_id_rel = pd.read_csv('store_id_relation.csv.zip')
df_sample = pd.read_csv('sample_submission.csv.zip')

### Some constants for later

In [3]:
# map months to seasons
seasons = {'Jan': 'Winter','Feb': 'Winter','Mar': 'Spring','Apr': 'Spring','May': 'Spring','Jun': 'Summer',
           'Jul': 'Summer','Aug': 'Summer','Sep': 'Autumn','Oct': 'Autumn','Nov': 'Autumn','Dec': 'Winter'}

# roll up genres
genres = {
    'Japanese style':'Japanese',
    'International cuisine':'Other',
    'Grilled meat':'Other Asian',
    'Creation':'Japanese',
    'Italian':'European',
    'Seafood':'Other',
    'Spain Bar/Italian Bar':'European',
    'Japanese food in general':'Japanese',
    'Shabu-shabu/Sukiyaki':'Japanese',
    'Chinese general':'Other Asian',
    'Creative Japanese food':'Japanese',
    'Japanese cuisine/Kaiseki':'Japanese',
    'Korean cuisine':'Other Asian',
    'Okonomiyaki/Monja/Teppanyaki':'Japanese',
    'Karaoke':'Bar or Club',
    'Steak/Hamburger/Curry':'Other',
    'French':'European',
    'Cafe':'European',
    'Bistro':'Other',
    'Sushi':'Japanese',
    'Party':'Bar or Club',
    'Western food':'Other',
    'Pasta/Pizza':'Other',
    'Thai/Vietnamese food':'Other Asian',
    'Bar/Cocktail':'Bar or Club',
    'Amusement bar':'Bar or Club',
    'Cantonese food':'Other Asian',
    'Dim Sum/Dumplings':'Other Asian',
    'Sichuan food':'Other Asian',
    'Sweets':'Other',
    'Spain/Mediterranean cuisine':'European',
    'Udon/Soba':'Japanese',
    'Shanghai food':'Other Asian',
    'Taiwanese/Hong Kong cuisine':'Other Asian',
    'Japanese food':'Japanese', 
    'Dining bar':'Bar or Club', 
    'Izakaya':'Japanese',
    'Okonomiyaki/Monja/Teppanyaki':'Japanese', 
    'Italian/French':'European', 
    'Cafe/Sweets':'Other',
    'Yakiniku/Korean food':'Other Asian', 
    'Western food':'Other', 
    'Bar/Cocktail':'Bar or Club', 
    'Other':'Other',
    'Creative cuisine':'Japanese', 
    'Karaoke/Party':'Bar or Club', 
    'International cuisine':'Other',
    'Asian':'Other Asian',
    'None':'None',
    'No Data':'No Data'}

# function to aggregate various values to add as features. Tables will be added to a dictionary
def groupings(df,group,field,agg,name,cols):
    for i in group:
        tmp = df.groupby([i])\
                .agg(agg)
        tmp.columns = tmp.columns.map(''.join)
        tmp.columns=[s + i for s in cols]
        tmp.reset_index(inplace=True)
        name[i]=tmp
        name[i].columns = name[i].columns.map(''.join)
        for j in field:
            tmp = df.groupby([i,j])\
                    .agg(agg)
            tmp.columns = tmp.columns.map(''.join)
            tmp.columns=[s + i+j for s in cols]
            tmp.reset_index(inplace=True)
            name[i+j] = tmp
            
# for the CV
splits=4
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

# calculate error
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

### Wrangle the reservation file

In [4]:
# merge 'air' tables and bring over any 'hpg' store data
df_air_merged = df_air_reserve.merge(df_air_store,on='air_store_id', how='left').merge(
    df_store_id_rel, on='air_store_id', how='left').merge(df_hpg_store,on='hpg_store_id', how='left',suffixes=('_air','_hpg'))

In [5]:
# merge 'hpg' tables and bring over any 'air' store data
df_hpg_merged = df_hpg_reserve.merge(df_hpg_store,on='hpg_store_id', how='left').merge(
    df_store_id_rel,on='hpg_store_id', how='left').merge(df_air_store,on='air_store_id', how='left',suffixes=('_hpg','_air'))

In [6]:
# add source column
df_air_merged['source'] = 'air'
df_hpg_merged['source'] = 'hpg'

In [7]:
# append tables together
df_res_merged = df_air_merged.append(df_hpg_merged, sort=True)
df_res_merged.reset_index(inplace=True)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [8]:
# format date fields
df_res_merged['visit_datetime'] = pd.to_datetime(df_res_merged.visit_datetime)
df_res_merged['reserve_datetime'] = pd.to_datetime(df_res_merged.reserve_datetime)

df_res_merged['calendar_date'] = df_res_merged.visit_datetime.dt.date
df_res_merged['visit_time'] = df_res_merged.visit_datetime.dt.time
df_res_merged['reserve_date'] = df_res_merged.reserve_datetime.dt.date
df_res_merged['reserve_time'] = df_res_merged.reserve_datetime.dt.time

In [9]:
# add month, year, and season
df_res_merged['visit_month'] = df_res_merged.visit_datetime.apply(lambda x: x.strftime("%b"))
df_res_merged['visit_year'] = df_res_merged.visit_datetime.apply(lambda x: x.strftime("%Y"))
df_res_merged['reserve_month'] = df_res_merged.reserve_datetime.apply(lambda x: x.strftime("%b"))
df_res_merged['reserve_year'] = df_res_merged.reserve_datetime.apply(lambda x: x.strftime("%Y"))

df_res_merged['reserve_season'] = df_res_merged['reserve_month'].map(seasons)
df_res_merged['visit_season'] = df_res_merged['visit_month'].map(seasons)

In [10]:
# format df_date_info date to merge
df_date_info['calendar_date'] = pd.to_datetime(df_date_info.calendar_date)
df_date_info['calendar_date'] = df_date_info.calendar_date.dt.date
df_res_merged = df_res_merged.merge(df_date_info, on='calendar_date', how='left')
df_res_merged.rename(columns={"day_of_week": "day_of_week_visit", "holiday_flg": "holiday_flag_visit"}, inplace=True)
df_res_merged = df_res_merged.merge(df_date_info, left_on='reserve_date', right_on='calendar_date', how='left')
df_res_merged.rename(columns={"day_of_week": "day_of_week_res", "holiday_flg": "holiday_flag_res","calendar_date_x": "visit_date"}, inplace=True)

In [11]:
# time between reservation and visit
df_res_merged['res_vs_visit'] = df_res_merged['visit_datetime'] - df_res_merged['reserve_datetime']
df_res_merged['res_vs_visit_days'] = df_res_merged['res_vs_visit'].astype('timedelta64[D]')
df_res_merged['res_vs_visit_hours'] = df_res_merged['res_vs_visit'].astype('timedelta64[h]')

In [12]:
# holiday the day before and after visit
df_res_merged['holiday_before_visit'] = df_res_merged.holiday_flag_visit.shift(1)
df_res_merged.holiday_before_visit.fillna(0,inplace=True)
df_res_merged['holiday_after_visit'] = df_res_merged.holiday_flag_visit.shift(-1)
df_res_merged.holiday_after_visit.fillna(0,inplace=True)

In [13]:
# too many genres - amalgamate

df_res_merged.hpg_genre_name.fillna('No Data', inplace=True)
df_res_merged.air_genre_name.fillna('No Data', inplace=True)
df_res_merged.hpg_store_id.fillna('No Data', inplace=True)
df_res_merged.air_store_id.fillna('No Data', inplace=True)
df_res_merged.hpg_area_name.fillna('No Data', inplace=True)
df_res_merged.air_area_name.fillna('No Data', inplace=True)
df_res_merged['air_genre_amal'] = df_res_merged['air_genre_name'].map(genres)
df_res_merged['hpg_genre_amal'] = df_res_merged['hpg_genre_name'].map(genres)

# amalgamated genre - take air genre first then hpg
df_res_merged['genre_amal']=df_res_merged['air_genre_amal']
df_res_merged.loc[df_res_merged['air_genre_amal']=='No Data',['genre_amal']] = df_res_merged['hpg_genre_amal']

In [14]:
# original genre - take air genre first then hpg
df_res_merged['genre_2']=df_res_merged['air_genre_name']
df_res_merged.loc[df_res_merged['air_genre_name']=='No Data',['genre_2']] = df_res_merged['hpg_genre_name']

In [15]:
# area - take air first then hpg
df_res_merged['area_2']=df_res_merged['air_area_name']
df_res_merged.loc[df_res_merged['air_area_name']=='No Data',['area_2']] = df_res_merged['hpg_area_name']

In [16]:
# get prefecture from area
df_res_merged['area_2_pref'] = df_res_merged.area_2.apply(lambda x: x.split(' ')[0] if pd.notnull(x) else x)
df_res_merged.loc[(df_res_merged['area_2_pref']=='No') | (df_res_merged['area_2_pref']=='None'),['area_2_pref']] = 'No Data'

### Wrangle the train/test files

In [17]:
# prep sample submission df
df_sample['visit_date'] = df_sample['id'].map(lambda x: str(x).split('_')[2])
df_sample['air_store_id'] = df_sample['id'].map(lambda x: '_'.join(x.split('_')[:2]))

In [18]:
# append visit and sample file to consistently format features
df_features=df_air_visit.append(df_sample, sort=True)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [19]:
# format df_date_info and add month, year, and season

df_features['visit_date'] = pd.to_datetime(df_features.visit_date)
df_features['visit_date'] = df_features.visit_date.dt.date
df_features = df_features.merge(df_date_info, left_on='visit_date', right_on='calendar_date', how='left')
df_features.drop('calendar_date', axis=1, inplace=True)
df_features = df_features.merge(df_air_store,on='air_store_id', how='left')
df_features['visit_month'] = df_features.visit_date.apply(lambda x: x.strftime("%b"))
df_features['visit_year'] = df_features.visit_date.apply(lambda x: x.strftime("%Y"))
df_features['visit_season'] = df_features['visit_month'].map(seasons)

In [20]:
# amalgamate genres like in reservations
df_features.air_genre_name.fillna('No Data', inplace=True)
df_features['air_genre_2'] = df_features['air_genre_name'].map(genres)

In [21]:
# holiday the day before and after visit
df_features['holiday_before_visit'] = df_features.holiday_flg.shift(1)
df_features.holiday_before_visit.fillna(0,inplace=True)
df_features['holiday_after_visit'] = df_features.holiday_flg.shift(-1)
df_features.holiday_after_visit.fillna(0,inplace=True)

In [22]:
# prefecture from area
df_features['area_pref'] = df_features.air_area_name.apply(lambda x: x.split(' ')[0] if pd.notnull(x) else x)

In [23]:
# add golden week
df_features['golden_week'] = df_features.visit_date.apply(lambda x: 1 if 
                            ((x>=(datetime.date(2016,4,29))) & (x<=(datetime.date(2016,5,7)))) |
                            ((x>=(datetime.date(2017,4,29))) & (x<=(datetime.date(2017,5,7))))
                                                                        else 0)

In [24]:
# fix outliers - they dont appear to be an organic spike in visitors
df_features.loc[df_features.visitors>250,['visitors']]= df_features.visitors//10

In [25]:
df_features['weekday'] = df_features.visit_date.apply(lambda x: x.weekday())
df_features['month'] = df_features.visit_date.apply(lambda x: x.month)
df_features['weekend'] = df_features.weekday.apply(lambda x: 1 if x>=5 else 0)
df_features['visit_year'] = df_features.visit_year.astype(int)
df_features['holiday_flg'] = df_features.holiday_flg.astype(float)

In [26]:
# copy original air_visit back to df_air_visit dataframe before calcuating aggregations
df_air_visit = df_features[df_features.id.isna()].copy()

# only include entries after 01 Jul 2016 as this is when we observe a significant and consistent uptick in values
date_incl = datetime.date(2016,7,1)
df_air_visit_incl = df_air_visit[(df_air_visit.visit_date)>=(date_incl)]
df_res_merged_incl = df_res_merged[(df_res_merged.visit_date)>=(date_incl)]

In [27]:
# aggregate df_air_visit dataframe
grp=['air_store_id','air_genre_2','air_genre_name','air_area_name','area_pref']
fld=['visit_month','visit_season','day_of_week','holiday_flg','holiday_before_visit','holiday_after_visit','golden_week','weekend']
aggr={'visitors':['mean','min','max','size']}
col=['v_mean_','v_min_','v_max_','v_cnt']
df_air_visit_grp={}
groupings(df_air_visit_incl,grp,fld,aggr,df_air_visit_grp,col)

In [28]:
# aggregate df_res_merged dataframe
grp=['genre_amal','genre_2','air_store_id','area_2_pref','area_2']
fld=['visit_month','visit_season','day_of_week_visit','holiday_flag_visit','holiday_before_visit','holiday_after_visit','visit_date']
aggr={'res_vs_visit_hours':'mean','reserve_visitors':'size'}
col=['rvvh_','rvs_']
df_res_merged_grp={}
groupings(df_res_merged_incl,grp,fld,aggr,df_res_merged_grp,col)

In [29]:
# add selected aggregates to features df and scale them
vis_grps = ['air_store_id', 'air_store_idvisit_month','air_store_idday_of_week', 
            'air_store_idholiday_flg','air_store_idholiday_after_visit',
            'area_prefday_of_week', 'air_genre_name', 'air_area_name',]

res_grps = ['air_store_id']

for i in df_air_visit_grp:
    if i in vis_grps:
        #scaler = preprocessing.RobustScaler()
        tmp = df_air_visit_grp[i]
        #cols = [col for col in tmp if col.startswith('v_')]
        #tmp[cols] = scaler.fit_transform(tmp[cols])
        df_features = df_features.merge(tmp,how='left')
        
for i in df_res_merged_grp:
    if i in res_grps:
        #scaler = preprocessing.RobustScaler()
        tmp = df_res_merged_grp[i]
        #cols = [col for col in tmp if col.startswith('rv')]
        #tmp[cols] = scaler.fit_transform(tmp[cols])
        df_features = df_features.merge(tmp,how='left')

In [30]:
df_features['visit_date_int'] = df_features.visit_date.apply(
    lambda x: x.strftime('%Y%m%d')).astype(int)
# Instance of day of the week in the month eg/ 2nd Monday etc
a = df_features.groupby(
    ['visit_year', 'visit_month', 'day_of_week',
     'visit_date_int'])['visit_date'].nunique().reset_index()
a['x'] = a.groupby(['visit_year', 'visit_month',
                    'day_of_week'])['visit_date'].cumsum()
a.set_index('visit_date_int', inplace=True)
df_features['visit_dow_cnt_month'] = df_features.visit_date_int.map(a['x'])

In [31]:
# Rank of store in area per day of week
a = df_air_visit_incl.groupby(
    ['air_area_name', 'day_of_week',
     'air_store_id'])['visitors'].sum().reset_index()
a.sort_values(by=['air_area_name', 'day_of_week', 'visitors'], inplace=True)
a['store_rank_area_dow'] = a.groupby(['air_area_name', 'day_of_week'
                                      ])['air_store_id'].cumcount() + 1
df_features = df_features.merge(
    a[['air_store_id', 'day_of_week', 'store_rank_area_dow']],
    how='left',
    on=['air_store_id', 'day_of_week'])

In [32]:
# Rank of store in pref per day of week
a = df_air_visit_incl.groupby(
    ['area_pref', 'day_of_week',
     'air_store_id'])['visitors'].sum().reset_index()
a.sort_values(by=['area_pref', 'day_of_week', 'visitors'], inplace=True)
a['store_rank_pref_dow'] = a.groupby(['area_pref', 'day_of_week'
                                      ])['air_store_id'].cumcount() + 1
df_features = df_features.merge(
    a[['air_store_id', 'day_of_week', 'store_rank_pref_dow']],
    how='left',
    on=['air_store_id', 'day_of_week'])

In [33]:
# Rank of store in area
a = df_air_visit_incl.groupby(
    ['air_area_name', 'air_store_id'])['visitors'].sum().reset_index()
a.sort_values(by=['air_area_name', 'visitors'], inplace=True)
a['store_rank_area'] = a.groupby(['air_area_name'
                                  ])['air_store_id'].cumcount() + 1
df_features = df_features.merge(
    a[['air_store_id', 'store_rank_area']], how='left', on=['air_store_id'])

In [34]:
# Rank of store in area per genre
a = df_air_visit_incl.groupby(
    ['air_area_name', 'air_genre_name',
     'air_store_id'])['visitors'].sum().reset_index()
a.sort_values(by=['air_area_name', 'air_genre_name', 'visitors'], inplace=True)
a['store_rank_area_genre'] = a.groupby(['air_area_name', 'air_genre_name'
                                        ])['air_store_id'].cumcount() + 1
df_features = df_features.merge(
    a[['air_store_id', 'air_genre_name', 'store_rank_area_genre']],
    how='left',
    on=['air_store_id', 'air_genre_name'])

In [35]:
# Rank of store in pref per genre
a = df_air_visit_incl.groupby(
    ['area_pref', 'air_genre_name',
     'air_store_id'])['visitors'].sum().reset_index()
a.sort_values(by=['area_pref', 'air_genre_name', 'visitors'], inplace=True)
a['store_rank_pref_genre'] = a.groupby(['area_pref', 'air_genre_name'
                                        ])['air_store_id'].cumcount() + 1
df_features = df_features.merge(
    a[['air_store_id', 'air_genre_name', 'store_rank_pref_genre']],
    how='left',
    on=['air_store_id', 'air_genre_name'])

In [36]:
# Feature interactions
df_features['lat_plus_long'] = df_features.latitude + df_features.longitude
df_features['lat_sub_max'] = df_features.latitude.max() - df_features.latitude
df_features['long_sub_max'] = df_features.longitude.max() - df_features.longitude
df_features['meanvis_sub_max'] = df_features.v_mean_air_store_id.max() - df_features.v_mean_air_store_id
df_features['meanres_sub_max'] = df_features.rvs_air_store_id.max() - df_features.rvs_air_store_id
df_features['meanvisdow_mul_cnt'] = df_features.v_cntair_store_idday_of_week * df_features.v_mean_air_store_idday_of_week
df_features['meanvis_mul_cnt'] = df_features.v_cntair_store_id * df_features.v_mean_air_store_id
df_features['meanresh_mul_cnt'] = df_features.rvvh_air_store_id * df_features.rvs_air_store_id
df_features['visit_date_int_mul_mean'] = df_features.visit_date_int * df_features.v_mean_air_store_id
df_features['meanvisdow_mul_rnkarea'] = df_features.v_cntair_store_idday_of_week * df_features.store_rank_area_dow
df_features['meanvisdow_mul_rnkpref'] = df_features.v_cntair_store_idday_of_week * df_features.store_rank_pref_dow

In [37]:
# some of the features in the test set have na...work out the mean from the train set to add down below
any_nulls = df_features.isna().any()
any_nulls = list(any_nulls[any_nulls==True].index)
mean_nulls = df_features[df_features.id.notna()].loc[:,any_nulls].mean()

In [38]:
# exclude dummies for:
# - air_store_id as mean per store is unique per store anyway.
# - season as information is included in the month
# - area_name as info included in lat/long
# - genre2 as info in air_genre_name

# df_features_dum = pd.get_dummies(df_features[(df_features.visit_date)>=(datetime.date(2016,7,1))],
#                                  columns=['air_genre_name','area_pref'],
#                                  drop_first=True)
# drop_cols = ['id','visit_date','visitors','air_store_id','air_area_name',
#              'visit_season','air_genre_2','day_of_week','visit_month','golden_week'


df_features_dum = df_features[(df_features.visit_date) >= (date_incl)].copy()
drop_cols = ['id','visit_date','visitors','air_store_id','air_area_name','air_genre_name','area_pref',
             'visit_season','air_genre_2','day_of_week','visit_month','golden_week']

df_train = df_features_dum[df_features_dum.id.isna()].sort_values(by='visit_date_int').copy()
y=np.log1p(df_train['visitors'].values)
df_train.drop(columns=drop_cols,inplace=True)

df_test = df_features_dum[df_features_dum.id.notna()].sort_values(by='visit_date_int').copy()
df_test.drop(columns=drop_cols,inplace=True)

df_test.fillna(-1,inplace=True)
df_train.fillna(-1,inplace=True)

#y=np.log1p(df_air_visit_incl['visitors'].values)

In [39]:
# LightGBM
tscv = model_selection.KFold(n_splits=splits)
tscv_cv = tscv.split(df_train)
params = {  
    "n_estimators": st.randint(10, 100),
    "learning_rate": st.uniform(0.05, 0.4),
    "num_leaves": st.randint(31, 100),
    "max_bin": st.randint(200, 400),
    "min_child_weight": st.randint(1, 10),
    "min_child_samples": st.randint(1, 100),
}   
lgbm = lgb.LGBMRegressor()
lgbmrscv = model_selection.RandomizedSearchCV(lgbm, params, n_iter=10,cv=tscv_cv)  
lgbmrscv.fit(df_train, y)
print(lgbmrscv.best_params_)
print('RMSE LGBMRegressor: ', RMSLE(y, lgbmrscv.predict(df_train)))
df_test_out = df_test.copy()
df_test_out['visitors'] = lgbmrscv.predict(df_test)
df_test_out['visitors'] = np.expm1(df_test_out['visitors']).clip(lower=1.)
df_test_out = df_test_out.sort_index()
df_sample['visitors'] = df_test_out['visitors'].values
lgbmrscv_out = df_sample[['id','visitors']].copy()
lgbmrscv_out.to_csv('lgbmrscv_out_'+datetime.datetime.now().strftime('%Y%m%d%I%M')+'.csv',index=False)

{'learning_rate': 0.132870537505463, 'max_bin': 316, 'min_child_samples': 54, 'min_child_weight': 6, 'n_estimators': 99, 'num_leaves': 69}
RMSE LGBMRegressor:  0.4486522607431908


In [40]:
# LightGBM with Time Series CV
tscv = model_selection.KFold(n_splits=splits)
tscv_cv = tscv.split(df_train)
y_test_pred = 0
lgbmtscv = lgbmrscv.best_estimator_
for i, (train_index, test_index) in enumerate(tscv.split(df_train)):
    # Create data for this fold
    y_train, y_valid = y[train_index].copy(), y[test_index]
    X_train, X_valid = df_train.iloc[train_index, :].copy(), df_train.iloc[test_index, :].copy()
    print("\nFold ", i)

    evals=[(X_valid,y_valid)]
    lgbmtscv.fit(X_train, y_train,eval_set=evals,early_stopping_rounds=5,verbose=False,eval_metric='l2_root')
    pred = lgbmtscv.predict(X_valid,num_iteration=lgbmtscv.best_iteration_)
    print('RMSE LightGBM best iteration, fold ', i, ': ', lgbmtscv.best_iteration_)
    print('RMSE LightGBM, fold ', i, ': ', RMSLE(y_valid, pred))
    # Accumulate test set predictions

    pred = lgbmtscv.predict(df_test,num_iteration=lgbmtscv.best_iteration_)
    y_test_pred += pred

y_test_pred/=splits
df_test_out = df_test.copy()
df_test_out['visitors'] = y_test_pred
df_test_out['visitors'] = np.expm1(df_test_out['visitors']).clip(lower=1.)
df_test_out = df_test_out.sort_index()
df_sample['visitors'] = df_test_out['visitors'].values
lgbmtscv_out = df_sample[['id','visitors']].copy()
lgbmtscv_out.to_csv('lgbmtscv_out_'+datetime.datetime.now().strftime('%Y%m%d%I%M')+'.csv',index=False)


Fold  0
RMSE LightGBM best iteration, fold  0 :  55
RMSE LightGBM, fold  0 :  0.47456850675712026

Fold  1
RMSE LightGBM best iteration, fold  1 :  70
RMSE LightGBM, fold  1 :  0.47042579691806985

Fold  2
RMSE LightGBM best iteration, fold  2 :  64
RMSE LightGBM, fold  2 :  0.4832980196150528

Fold  3
RMSE LightGBM best iteration, fold  3 :  39
RMSE LightGBM, fold  3 :  0.4635961685377659


In [41]:
# XGBRegressor
tscv = model_selection.KFold(n_splits=splits)
tscv_cv = tscv.split(df_train)
params = {  
    "n_estimators": st.randint(3, 40),
    "max_depth": st.randint(3, 40),
    "learning_rate": st.uniform(0.05, 0.4),
    "colsample_bytree": one_to_left,
    "subsample": one_to_left,
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,
    "min_child_weight": from_zero_positive,
}

xgbr = xgb.XGBRegressor()
xgbrscv = model_selection.RandomizedSearchCV(
    xgbr, params, n_iter=10, cv=tscv_cv)
xgbrscv.fit(df_train, y)
print(xgbrscv.best_params_)
print('RMSE XGBRegressor: ',
      RMSLE(y, xgbrscv.best_estimator_.predict(df_train)))
df_test_out = df_test.copy()
df_test_out['visitors'] = xgbrscv.best_estimator_.predict(df_test)
df_test_out['visitors'] = np.expm1(df_test_out['visitors']).clip(lower=1.)
df_test_out = df_test_out.sort_index()
df_sample['visitors'] = df_test_out['visitors'].values
xgbrscv_out = df_sample[['id', 'visitors']].copy()
xgbrscv_out.to_csv(
    'xgbrscv_out_' + datetime.datetime.now().strftime('%Y%m%d%I%M') + '.csv',
    index=False)

{'colsample_bytree': 0.9531025392538498, 'gamma': 8.387351870014246, 'learning_rate': 0.394083536041345, 'max_depth': 38, 'min_child_weight': 156.89985691533957, 'n_estimators': 16, 'reg_alpha': 10.549561598021711, 'subsample': 0.9652230734993309}
RMSE XGBRegressor:  0.46837784025257445


In [42]:
# XGBRegressor with Time Series CV
tscv = model_selection.KFold(n_splits=splits)
tscv_cv = tscv.split(df_train)
y_test_pred = 0
xgbtscv = xgbrscv.best_estimator_
for i, (train_index, test_index) in enumerate(tscv.split(df_train)):
    # Create data for this fold
    y_train, y_valid = y[train_index].copy(), y[test_index]
    X_train, X_valid = df_train.iloc[train_index, :].copy(), df_train.iloc[test_index, :].copy()
    print("\nFold ", i)

    evals = [(X_valid, y_valid)]
    xgbtscv.fit(
        X_train,
        y_train,
        eval_set=evals,
        early_stopping_rounds=5,
        verbose=False,
        eval_metric='rmse')
    pred = xgbtscv.predict(X_valid)
    print('RMSE XGBRegressor, fold ', i, ': ', RMSLE(y_valid, pred))
    print('RMSE XGBRegressor best iteration, fold ', i, ': ',
          xgbtscv.best_iteration)
    # Accumulate test set predictions

    pred = xgbtscv.predict(df_test)
    y_test_pred += pred

y_test_pred /= splits
df_test_out = df_test.copy()
df_test_out['visitors'] = y_test_pred
df_test_out['visitors'] = np.expm1(df_test_out['visitors']).clip(lower=1.)
df_test_out = df_test_out.sort_index()
df_sample['visitors'] = df_test_out['visitors'].values
xgbtscv_out = df_sample[['id', 'visitors']].copy()
xgbtscv_out.to_csv(
    'xgbtscv_out_' + datetime.datetime.now().strftime('%Y%m%d%I%M') + '.csv',
    index=False)


Fold  0
RMSE XGBRegressor, fold  0 :  0.48211208816076656
RMSE XGBRegressor best iteration, fold  0 :  15

Fold  1
RMSE XGBRegressor, fold  1 :  0.4774096833843761
RMSE XGBRegressor best iteration, fold  1 :  15

Fold  2
RMSE XGBRegressor, fold  2 :  0.4910563861068349
RMSE XGBRegressor best iteration, fold  2 :  15

Fold  3
RMSE XGBRegressor, fold  3 :  0.46774997966283244
RMSE XGBRegressor best iteration, fold  3 :  15


In [43]:
sub=pd.DataFrame()
sub=xgbtscv_out.merge(xgbrscv_out, on='id',suffixes=('_xgbtscv','_xgbrscv'))\
               .merge(lgbmtscv_out,on='id')\
               .merge(lgbmrscv_out,on='id',suffixes=('lgbmtscv','lgbmrscv'))
df_sample['visitors'] = sub.mean(axis=1)
mean_out = df_sample[['id','visitors']].copy()
mean_out.to_csv('mean_out_'+datetime.datetime.now().strftime('%Y%m%d%I%M')+'.csv',index=False)

In [44]:
sub1 = pd.DataFrame()
sub1 = xgbtscv_out.merge(xgbrscv_out, on='id', suffixes=('xgbtscv', 'xgbrscv'))
df_sample['visitors'] = sub1.mean(axis=1)
mean_out1 = df_sample[['id', 'visitors']].copy()
mean_out1.to_csv(
    'mean_xgb_out_' + datetime.datetime.now().strftime('%Y%m%d%I%M') + '.csv',
    index=False)

In [45]:
sub2 = pd.DataFrame()
sub2 = lgbmtscv_out.merge(
    lgbmrscv_out, on='id', suffixes=('lgbmtscv', 'lgbmrscv'))
df_sample['visitors'] = sub2.mean(axis=1)
mean_out2 = df_sample[['id', 'visitors']].copy()
mean_out2.to_csv(
    'mean_lgbm_out_' + datetime.datetime.now().strftime('%Y%m%d%I%M') + '.csv',
    index=False)

In [46]:
sub3 = pd.DataFrame()
sub3 = xgbtscv_out.merge(
    lgbmtscv_out, on='id', suffixes=('xgbtscv', 'lgbmtscv'))
df_sample['visitors'] = sub3.mean(axis=1)
mean_out3 = df_sample[['id', 'visitors']].copy()
mean_out3.to_csv(
    'mean_tscv_out_' + datetime.datetime.now().strftime('%Y%m%d%I%M') + '.csv',
    index=False)